<a href="https://colab.research.google.com/github/roni762583/bible-data-science.github.io/blob/master/CleanBible.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import math


In [3]:
#clone data into datafile
! git clone https://github.com/roni762583/bible-data-science.github.io.git
# copy my datafile
! cp /content/bible-data-science.github.io/data/t3utf.dat datafile
# delete unused parts
! rm -rf /content/bible-data-science.github.io/
! ls -la

Cloning into 'bible-data-science.github.io'...
remote: Enumerating objects: 56, done.
remote: Counting objects: 100% (56/56), done.
remote: Compressing objects: 100% (56/56), done.
remote: Total 208 (delta 30), reused 0 (delta 0), pack-reused 152
Receiving objects: 100% (208/208), 7.94 MiB | 9.73 MiB/s, done.
Resolving deltas: 100% (107/107), done.
total 5648
drwxr-xr-x 1 root root    4096 Feb  1 23:09 .
drwxr-xr-x 1 root root    4096 Feb  1 23:05 ..
drwxr-xr-x 1 root root    4096 Jan 28 17:15 .config
-rw-r--r-- 1 root root 5763655 Feb  1 23:09 datafile
drwxr-xr-x 1 root root    4096 Jan 20 17:27 sample_data


In [8]:
# get unicode codes for each char in file preserving order and lines stucture 01/25/2021
f = 'א'
ef = f.encode("unicode_escape")
df = ef.decode("unicode_escape")
print(f,ef,df)
print('-----')
file = open('datafile','r')
lineCount = 0
NonHebrewCharsLst = list() # list to hold non-Hebrew block chars
badLines = list() # list for line numbers with non-Hebrew chars to manually review
exceptions = [' ']
nonHebrewLtrCt = 0 # loop counter
for line in file: 
    lineCount += 1
    index = line.rfind('|')+1
    verse = line[:index] # from begining to last occurance of | symbol
    lineText = line[index:].strip()#.replace('׃','') #strip removes newline char, replace removes sof-pasuk colon 
    
    # range of character code points in unicode Hebrew Block: 0590..05FF; Hebrew
    for char in lineText:
      hexS = hex(ord(char)) # string type, holding hex code of integer representing the Unicode character.
      intS = str(int(hexS,16)).encode("unicode_escape").decode("unicode_escape") # string type, int representing  the Unicode character
      # print('block start: ', int('0x590',16), 'char: ', int(hexS,16), 'block end: ', int('0x5FF',16))
      #break
      bol1 = int(hexS,16)>=int('0x590',16)
      bol2 = int(hexS,16)<=int('0x5FF',16)
      bol3 = bol1 and bol2
      # print(bol3)
      if (bol3): #i.e. if in Hebrew char block
        continue
        print(chr(int(hexS,16))) # hex code for char in
      else: # if char not in Hebrew block of unicode
        if (char in exceptions):
          continue
        nonHebrewLtrCt += 1
        NonHebrewCharsLst.append(intS) # str(int(hexS,16)))
        badLines.append(lineCount)
    if(not(bol3)):
      print('on line ', lineCount, 'look for ', line)
      cleanLine = lineText.replace('׃','') # replace removes sof-pasuk colon 
      kri = cleanLine
# output results
print('line count: ', lineCount)
print('verse: ', verse)
print('lineText: ', lineText)
encodedVerse = lineText.encode("unicode_escape")
decodedVerse = encodedVerse.decode("unicode_escape")
print('code points: ', encodedVerse)
print('decoded verse: ', decodedVerse)
uniqueChars = list(set(lineText)) # get unique chars list
print('unique chars: ', uniqueChars)

uniqueNonHebrewCharsLst = list(set(NonHebrewCharsLst))
print('number of unique non-Hebrew block chars', len(uniqueNonHebrewCharsLst))
# print out total count of non-Hebrew block chars, and unique set of charsof this count
print('nonHebrewLtrCt: ', nonHebrewLtrCt, '\nunique list: ', uniqueNonHebrewCharsLst)
# dictionary for line numbers of non-Hebrew occurances
dic = dict()
for c in uniqueNonHebrewCharsLst:
  #print(chr(int(c)))
  if chr(int(c)) in dic:
    dic[chr(int(c))] = dic[chr(int(c))] + 1
  else:
    dic[chr(int(c))] = 1
# print dictionary
#print('dictionary of non-Heb char ct: ', dic)
#print(int('0x5FF',16))
#print(int(hexS,16))
#print()
print(set(badLines))
print(badLines[0])
#print(len(s))


א b'\\u05d0' א
-----
on line  731 look for  gen|27|3|וְעַתָּה֙ שָׂא־נָ֣א כֵלֶ֔יךָ תֶּלְיְךָ֖ וְקַשְׁתֶּ֑ךָ וְצֵא֙ הַשָּׂדֶ֔ה וְצ֥וּדָה לִּ֖י <font color="brown">צֵידָה</font> <font color="blue">צָֽיִד׃</font> 

on line  1319 look for  gen|43|28|וַיֹּאמְר֗וּ שָׁלֹ֛ום לְעַבְדְּךָ֥ לְאָבִ֖ינוּ עֹודֶ֣נּוּ חָ֑י וַֽיִּקְּד֖וּ <font color="brown">וַיִּשְׁתַּחוּ</font> <font color="blue">וַיִּֽשְׁתַּחֲוּֽוּ<font color="red"><sup>1</font></sup>׃</font> 

on line  1485 look for  gen|49|11|אֹסְרִ֤י לַגֶּ֙פֶן֙ <font color="brown">עִירֹה</font> <font color="blue">עִירֹ֔ו</font> וְלַשֹּׂרֵקָ֖ה בְּנִ֣י אֲתֹנֹ֑ו כִּבֵּ֤ס בַּיַּ֙יִן֙ לְבֻשֹׁ֔ו וּבְדַם־עֲנָבִ֖ים <font color="brown">סוּתֹה</font> <font color="blue">סוּתֹֽו׃</font> 

on line  2613 look for  exo|37|8|כְּרוּב־אֶחָ֤ד מִקָּצָה֙ מִזֶּ֔ה וּכְרוּב־אֶחָ֥ד מִקָּצָ֖ה מִזֶּ֑ה מִן־הַכַּפֹּ֛רֶת עָשָׂ֥ה אֶת־הַכְּרֻבִ֖ים מִשְּׁנֵ֥י <font color="brown">קִצְוֹותֹו</font> <font color="blue">קְצֹותָֽיו׃</font> 

on line  4023 look for  num|10|34|וַעֲנַ֧ן יְ

In [9]:
# Build Tanach string without non-Hebrew letters characters
# Opening file 
file1 = open('datafile', 'r') 
count = 0
s = '' # basic Hebrew chars string
s2 = '' # non-Hebrew chars string
# dataframe for reshaped data
# df = pd.dataframe()
# loop over file line by line, one verse per line 
for line in file1: 
    count += 1
    #print("Line{}: {}".format(count, line.strip())) 
    #print(line[0]) # first character in line
    #print(line.rfind('|')) # last index of pipe '|' character
    line = (line[line.rfind('|')+1:]).replace('׃','') #remove verse and sof-pasuk colon
    line = line.strip() # remove newline char
    line = line.replace(" ", "") # remove spaces
    for c in line:
      if c in {'א', 'ב', 'ג' ,'ד','ה','ו','ז','ח','ט','י','כ','ך','ל','מ','ם','נ','ן','ס','ע','פ','ף','צ','ץ','ק','ר','ש','ת'}:
        s += c # concat Hebrew letter to s
      else:
        s2 += c
print (list(set(s))) # get unique chars list
print (s[-1]) # print last letter ל
# Closing files 
file1.close()

['ס', 'מ', 'ד', 'ל', 'ג', 'כ', 'ח', 'ש', 'ב', 'ן', 'ק', 'ך', 'פ', 'ו', 'ת', 'ה', 'ר', 'ץ', 'ע', 'י', 'ם', 'צ', 'ז', 'א', 'נ', 'ט', 'ף']
ל


In [10]:
# Pull off els substrings, offset strings = els

def findTermIndeces(text, term):
  # print('hello from findTermIndeces()')
  indxLst = list()
  indx = text.find(term) # index of first occurance of term
  while indx != -1: # while term is found in text
    indxLst.append(indx) # add index of found term to index list
    indx = text.find(term,indx+1) # find next occurance of search term after index of prev. find 
  return indxLst

def searchELS(text, term, minELS, maxELS):
  # print('hello from searchELS()')
  # initially address positive ELS values only
  # ternary-operator
  minELS = minELS if minELS>0 else 1
  foundTermIndeces = list() # empty list to hold indeces of found search term
  results = dict()
  elsRng = range(minELS, maxELS)
  for els in elsRng:
    # print('searching ELS = ', els)
    # get ELS substring with offset from 0 to els (current els being examined)
    # n = ELS substrings required to cover search space
    for offset in range(0, els): # get all offsets from 0 to els
      shiftedString = text[offset::els] # get offset string
      foundTermIndeces = foundTermIndeces + findTermIndeces(shiftedString, term) # append new list of indeces to exisiting built list
    if(els>0): # els 0 is non-sensical, open text is els = 1 here
      results[els] = foundTermIndeces # add found indeces to dictionary keyed by els
    foundTermIndeces = list() # clean out old list
  return results
# ex:
# print(searchELS(s,'עדנה', 0, 17))

In [13]:
# get user input for search term
term1 = input ("Enter Primary Search Term:")
print(term1)


Enter Primary Search Term:עדנה
עדנה
{1: [21078, 108600, 574590, 835684, 1019725, 1169894], 2: [537386, 226804, 435273], 3: [381772], 4: [171879, 173530], 5: [49034, 187730, 220698, 6005, 89354, 150583, 2483, 125792, 181036], 6: [105530, 84523, 183031, 107340, 68631, 98099], 7: [70086, 42814], 8: [31961, 103991], 9: [3905, 11181, 22953, 128010, 54879], 10: [85565], 11: [104380, 30806, 85452, 3043, 58358], 12: [16976, 82976], 13: [20181, 45770, 21414, 27347], 14: [41816, 40136, 7700], 15: [21454, 72275, 64447], 16: [29015, 43909, 44132, 30128], 17: [12806, 18167, 14129], 18: [6276, 39172, 32216], 19: [55605, 10812, 22595, 55995], 20: [38973, 24854, 53085, 7863, 23067], 21: [10088], 22: [45864, 1242, 27304, 5534], 23: [8389], 24: [45254, 37382], 25: [45635], 26: [23327], 27: [10489, 39661], 28: [3016, 38236, 41552, 25309, 28788], 29: [30250, 4418, 6413, 30576, 39217, 18859, 1158], 30: [], 31: [26771, 14242, 8244, 27841, 28128, 15785, 2881, 2047], 32: [26995, 17096, 29109, 18843, 30310, 29

In [14]:
print(searchELS(s, term1, 0, 170))

{1: [21078, 108600, 574590, 835684, 1019725, 1169894], 2: [537386, 226804, 435273], 3: [381772], 4: [171879, 173530], 5: [49034, 187730, 220698, 6005, 89354, 150583, 2483, 125792, 181036], 6: [105530, 84523, 183031, 107340, 68631, 98099], 7: [70086, 42814], 8: [31961, 103991], 9: [3905, 11181, 22953, 128010, 54879], 10: [85565], 11: [104380, 30806, 85452, 3043, 58358], 12: [16976, 82976], 13: [20181, 45770, 21414, 27347], 14: [41816, 40136, 7700], 15: [21454, 72275, 64447], 16: [29015, 43909, 44132, 30128], 17: [12806, 18167, 14129], 18: [6276, 39172, 32216], 19: [55605, 10812, 22595, 55995], 20: [38973, 24854, 53085, 7863, 23067], 21: [10088], 22: [45864, 1242, 27304, 5534], 23: [8389], 24: [45254, 37382], 25: [45635], 26: [23327], 27: [10489, 39661], 28: [3016, 38236, 41552, 25309, 28788], 29: [30250, 4418, 6413, 30576, 39217, 18859, 1158], 30: [], 31: [26771, 14242, 8244, 27841, 28128, 15785, 2881, 2047], 32: [26995, 17096, 29109, 18843, 30310, 29721, 10337], 33: [9184, 6726, 8401],

In [17]:
results = searchELS(s, term1, 0, 3)
for k in results:
  for idx in results[k]:
    print(s[idx:idx+4])
# need to get correct offset to reconstruct

עדנה
עדנה
עדנה
עדנה
עדנה
עדנה
צואו
יהוה
להוד


In [ ]:
# write no spaces string to file - only run once
text_file = open("NoSpacesTanach.dat", "w")
text_file.write(s)
text_file.close()

# delete intermediary datafile
! rm datafile

#see
! ls -la

total 2368
drwxr-xr-x 1 root root    4096 Jan 17 05:33 .
drwxr-xr-x 1 root root    4096 Jan 17 02:30 ..
drwxr-xr-x 1 root root    4096 Jan  8 17:11 .config
-rw-r--r-- 1 root root 2405776 Jan 17 05:33 NoSpacesTanach.dat


In [ ]:
# download result file - only run once
from google.colab import files
files.download('NoSpacesTanach.dat')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>